In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('lyrics_save500.txt', sep='\t')

In [3]:
df

,Unnamed: 0,eng,kor
0,0,Time runs,시간은 달려가
1,1,"Like my dreams, day and night",마치 내 꿈처럼 day and night
2,2,I’m carrying them on my neck,잠시 내 목마를 태워
3,3,Throwing my memories outside,나의 추억들을 밖으로 던진다
4,4,This is like a drama without a script,이건 마치 각본 없는 drama
...,...,...,...
49584,20,"Today, I’m shining brightly",모르지만 오늘도 난 밝게 빛나지
49585,27,"Wherever they are, I’ll be the brightest",어디서 보아도 내가 제일 눈부시게
49586,28,"Even though time passes, I won’t wither away",시간이 지나도 사그라들지 않게
49587,29,"Even if I run out of breath, I won’t stop and ...",숨이 차 올라도 멈추지 말고 더 크게 외쳐 oh
